Only focus on the limited set of logs returns variabe.

In [6]:
import os, sys
sys.path.insert(1,"../")
from analysis_utils import *

pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

# I. Prepare Data
We want to make sure to incorporate **SUBSAMPLING** to make help with the lopsided distribution

In [57]:
d = pd.read_pickle("../Log_processed_data/ohlcv_data_training_h_log.pkl")
print("Hourly data",d.shape)

Hourly data (43506, 681)


In [58]:
# Convert exchange variable to categorical
d['exchange_cat'] = d.exchange.astype('category').cat.codes

In [59]:
d.filter(regex="sum|log").columns.sort_values()

Index(['log_12h_high', 'log_12h_open', 'log_12h_volumefrom',
       'log_12h_volumeto', 'log_24h_high', 'log_24h_open',
       'log_24h_volumefrom', 'log_24h_volumeto', 'log_36h_high',
       'log_36h_open',
       ...
       'sum_24h_volumefrom', 'sum_24h_volumeto', 'sum_36h_volumefrom',
       'sum_36h_volumeto', 'sum_3h_volumefrom', 'sum_3h_volumeto',
       'sum_6h_volumefrom', 'sum_6h_volumeto', 'sum_9h_volumefrom',
       'sum_9h_volumeto'],
      dtype='object', length=374)

Features to include:

In [60]:
#HOURLY LOG DATA: GOES BACK 6 HOURS
log_return = ['log_return_h'+str(x) for x in range(1,7)]
log_high = ['log_high_h'+str(x) for x in range(1,7)]
log_volume = ['log_volumefrom_h'+str(x) for x in range(1,7)]
#SUM DATA
sum_log_open = ["log_"+str(x)+"h_open" for x in [3,6,9,12,24,36] ]
sum_log_high = ["log_"+str(x)+"h_high" for x in [3,6,9,12,24,36] ]
sum_volumefrom = ['sum_'+str(x)+'h_volumefrom'for x in [3,6,9,12,24,36]]
sum_volumeto = ['sum_'+str(x)+'h_volumeto'for x in [3,6,9,12,24,36]]
base_feats = ['channelLink','channelTitle','currency','exchange','coin_name','timestamp']
#MISC. DATA
time_data = ['pump_year','pump_month','pump_weekday','pump_hour']

In [61]:
features =  ['exchange_cat'] + time_data +  log_return + log_high + log_volume + sum_log_open + sum_log_high + sum_volumefrom + sum_volumeto

In [62]:
#We will look at only Binance data
d = d[d.exchange == "Binance"]
print("There are {} data points in Binance".format(d.shape[0]))

There are 27306 data points in Binance


In [63]:
d[features + base_feats].head(3)

,exchange_cat,pump_year,pump_month,pump_weekday,pump_hour,log_return_h1,log_return_h2,log_return_h3,log_return_h4,log_return_h5,log_return_h6,log_high_h1,log_high_h2,log_high_h3,log_high_h4,log_high_h5,log_high_h6,log_volumefrom_h1,log_volumefrom_h2,log_volumefrom_h3,log_volumefrom_h4,log_volumefrom_h5,log_volumefrom_h6,log_3h_open,log_6h_open,log_9h_open,log_12h_open,log_24h_open,log_36h_open,log_3h_high,log_6h_high,log_9h_high,log_12h_high,log_24h_high,log_36h_high,sum_3h_volumefrom,sum_6h_volumefrom,sum_9h_volumefrom,sum_12h_volumefrom,sum_24h_volumefrom,sum_36h_volumefrom,sum_3h_volumeto,sum_6h_volumeto,sum_9h_volumeto,sum_12h_volumeto,sum_24h_volumeto,sum_36h_volumeto,channelLink,channelTitle,currency,exchange,coin_name,timestamp
1620,0,2018,9,0,14,0.001509,0.008671,0.005517,0.004376,0.000000,0.005135,-0.042622,-0.055959,0.067536,-0.152277,0.020203,0.029776,-0.357811,-1.860519,-0.880844,2.098907,-0.461861,-1.424360,-0.088144,-0.160175,-0.212819,-0.301558,-0.020928,-0.346350,-0.098581,-0.163119,-0.208015,-0.312155,-0.035402,-0.344590,4.740495e+12,1.665632e+13,3.328419e+13,5.542504e+13,8.183836e+13,1.149128e+14,3.402400e+08,1.160540e+09,2.475540e+09,4.424540e+09,6.346490e+09,9.069600e+09,https://t.me/joinchat/AAAAAETRfXVAYuIieZ7K6g,Crypto Trading.,DNT,Binance,SALT,1536588014
1621,0,2018,9,0,14,-0.008205,0.006651,0.008201,0.042803,0.008452,-0.003908,-0.070548,-0.049826,0.030153,-0.197207,0.023893,-0.064197,1.762726,-2.002445,-1.523415,3.129602,-0.828386,0.137518,-0.110150,-0.269645,-0.319104,-0.432671,-0.036112,-0.437299,-0.120374,-0.263535,-0.318915,-0.439728,-0.045643,-0.433824,2.987310e+13,1.114464e+14,1.648195e+14,2.004899e+14,3.908148e+14,4.708632e+14,3.113400e+08,1.179140e+09,1.919170e+09,2.431800e+09,4.891720e+09,5.875067e+09,https://t.me/joinchat/AAAAAETRfXVAYuIieZ7K6g,Crypto Trading.,DNT,Binance,POA,1536588014
1622,0,2018,9,0,14,0.001362,0.002592,-0.017822,0.018955,0.000796,-0.006936,-0.087687,-0.137911,0.015811,-0.229498,-0.062904,0.089380,-0.963875,0.803721,0.187596,0.801292,-0.077188,-1.410568,-0.233253,-0.478333,-0.227348,-0.113063,0.377785,0.105223,-0.225598,-0.502189,-0.272519,-0.107079,0.377781,0.112621,2.131180e+13,2.967190e+13,4.500500e+13,4.961640e+13,1.333806e+14,2.165655e+14,1.916800e+09,2.858000e+09,4.589300e+09,5.043890e+09,1.101049e+10,1.624072e+10,https://t.me/joinchat/AAAAAETRfXVAYuIieZ7K6g,Crypto Trading.,DNT,Binance,MDA,1536588014


## Spit Data
The __subsample__ function incoporates the sklearn.train_test_split_ function in the import section

In [462]:
x_train, x_test, y_train, y_test = subsample(d[features], d['pumped_yn'], num=15000)

Original :
x_train:(21844, 47), x_test:(5462, 47), y_train:(21844,), y_test:(5462,)
There are 191 positive samples in the original train set!
There are 55 positive samples in the original train set!
Subsample :
x_train:(15191, 47), y_train:(15191,), x_test:(5462, 47), y_test:(5462,)


## II. RANDOM FOREST

In [463]:
from sklearn.ensemble import RandomForestClassifier as RF

In [464]:
rf = RF()
params_rf = {'n_estimators':[50,100,200,500]
            ,'max_features': ['auto', 'sqrt', 'log2']
            ,'max_depth' : [10,15,25,50,75]
            ,'criterion' :['gini', 'entropy']}

In [465]:
CV_rfc = GridSearchCV(estimator=rf, param_grid=params_rf, cv=5, scoring='roc_auc')
CV_rfc.fit(x_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [467]:
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'log2',
 'n_estimators': 200}

In [468]:
rf_best = RF(criterion='gini',max_depth=10,max_features='sqrt',n_estimators=200)
rf_best.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## ON TRAIN RESULT

In [356]:
y_train_prob = rf_best.predict_proba(x_train)[:,1]
# y_train_prob[y_train_prob > 0.16]

In [427]:
y_train_pred = np.zeros_like(y_train)
y_train_pred[y_train_prob > 0.09] = 1

In [428]:
# sum(y_train_pred)

In [429]:
tn,fp,fn,tp = confusion_matrix(y_train,y_train_pred).ravel()

In [430]:
print(tn,fp,fn,tp)

14946 54 64 139


## ON TEST RESULT

In [361]:
y_test_prob = rf_best.predict_proba(x_test)[:,1]

In [434]:
y_test_pred = np.zeros_like(y_test)
y_test_pred[y_test_prob > 0.3] = 1

In [435]:
tn,fp,fn,tp = confusion_matrix(y_test,y_test_pred).ravel()

In [436]:
print(tn,fp,fn,tp)

5290 129 38 5


## Part III. 
Incorporate __hourly__ data to see improvement...

In [122]:
s = pd.read_pickle("../social_data_training_h.pkl")
print(s.shape)
s.head(3)

(80940, 1626)


,channelLink,channelTitle,currency,exchange,timestamp,signalTime,pumptime_edited,coin_name,coin_id,code_repo_stars_h0,code_repo_stars_h1,code_repo_stars_h2,code_repo_stars_h3,code_repo_stars_h4,code_repo_stars_h5,code_repo_stars_h6,code_repo_stars_h7,code_repo_stars_h8,code_repo_stars_h9,code_repo_stars_h10,code_repo_stars_h11,code_repo_stars_h12,code_repo_stars_h13,code_repo_stars_h14,code_repo_stars_h15,code_repo_stars_h16,code_repo_stars_h17,code_repo_stars_h18,code_repo_stars_h19,code_repo_stars_h20,code_repo_stars_h21,code_repo_stars_h22,code_repo_stars_h23,code_repo_stars_h24,code_repo_stars_h25,code_repo_stars_h26,code_repo_stars_h27,code_repo_stars_h28,code_repo_stars_h29,code_repo_stars_h30,code_repo_stars_h31,code_repo_stars_h32,code_repo_stars_h33,code_repo_stars_h34,code_repo_stars_h35,code_repo_stars_h36,code_repo_stars_h37,code_repo_stars_h38,code_repo_stars_h39,code_repo_stars_h40,code_repo_stars_h41,code_repo_stars_h42,code_repo_stars_h43,code_repo_stars_h44,code_repo_stars_h45,code_repo_stars_h46,code_repo_stars_h47,code_repo_stars_h48,code_repo_forks_h0,code_repo_forks_h1,code_repo_forks_h2,code_repo_forks_h3,code_repo_forks_h4,code_repo_forks_h5,code_repo_forks_h6,code_repo_forks_h7,code_repo_forks_h8,code_repo_forks_h9,code_repo_forks_h10,code_repo_forks_h11,code_repo_forks_h12,code_repo_forks_h13,code_repo_forks_h14,code_repo_forks_h15,code_repo_forks_h16,code_repo_forks_h17,code_repo_forks_h18,code_repo_forks_h19,code_repo_forks_h20,code_repo_forks_h21,code_repo_forks_h22,code_repo_forks_h23,code_repo_forks_h24,code_repo_forks_h25,code_repo_forks_h26,code_repo_forks_h27,code_repo_forks_h28,code_repo_forks_h29,code_repo_forks_h30,code_repo_forks_h31,code_repo_forks_h32,code_repo_forks_h33,code_repo_forks_h34,code_repo_forks_h35,code_repo_forks_h36,code_repo_forks_h37,code_repo_forks_h38,code_repo_forks_h39,code_repo_forks_h40,code_repo_forks_h41,code_repo_forks_h42,code_repo_forks_h43,code_repo_forks_h44,code_repo_forks_h45,code_repo_forks_h46,code_repo_forks_h47,code_repo_forks_h48,code_repo_subscribers_h0,code_repo_subscribers_h1,code_repo_subscribers_h2,code_repo_subscribers_h3,code_repo_subscribers_h4,code_repo_subscribers_h5,code_repo_subscribers_h6,code_repo_subscribers_h7,code_repo_subscribers_h8,code_repo_subscribers_h9,code_repo_subscribers_h10,code_repo_subscribers_h11,code_repo_subscribers_h12,code_repo_subscribers_h13,code_repo_subscribers_h14,code_repo_subscribers_h15,code_repo_subscribers_h16,code_repo_subscribers_h17,code_repo_subscribers_h18,code_repo_subscribers_h19,code_repo_subscribers_h20,code_repo_subscribers_h21,code_repo_subscribers_h22,code_repo_subscribers_h23,code_repo_subscribers_h24,code_repo_subscribers_h25,code_repo_subscribers_h26,code_repo_subscribers_h27,code_repo_subscribers_h28,code_repo_subscribers_h29,code_repo_subscribers_h30,code_repo_subscribers_h31,code_repo_subscribers_h32,code_repo_subscribers_h33,code_repo_subscribers_h34,code_repo_subscribers_h35,code_repo_subscribers_h36,code_repo_subscribers_h37,code_repo_subscribers_h38,code_repo_subscribers_h39,code_repo_subscribers_h40,code_repo_subscribers_h41,code_repo_subscribers_h42,code_repo_subscribers_h43,code_repo_subscribers_h44,code_repo_subscribers_h45,code_repo_subscribers_h46,code_repo_subscribers_h47,code_repo_subscribers_h48,code_repo_contributors_h0,code_repo_contributors_h1,code_repo_contributors_h2,code_repo_contributors_h3,code_repo_contributors_h4,code_repo_contributors_h5,code_repo_contributors_h6,code_repo_contributors_h7,code_repo_contributors_h8,code_repo_contributors_h9,code_repo_contributors_h10,code_repo_contributors_h11,code_repo_contributors_h12,code_repo_contributors_h13,code_repo_contributors_h14,code_repo_contributors_h15,code_repo_contributors_h16,code_repo_contributors_h17,code_repo_contributors_h18,code_repo_contributors_h19,code_repo_contributors_h20,code_repo_contributors_h21,code_repo_contributors_h22,code_repo_contributors_h23,code_repo_contributors_h24,code_repo_contributors_h25,code_